In [8]:
from openai import OpenAI
def run_chatgpt(api_key: str, prompt: list, history=[]):
    client = OpenAI(api_key=api_key)
    # history += prompt
    respond = client.chat.completions.create(
        messages=prompt,
        model="gpt-4o",
        temperature=0,
        # response_format= {"type":"json_object"},
    )
    # history.append({"role":"assistant", "content":respond.choices[0].message.content})
    # ans_JSON = []
    # ans_JSON.append(json.loads(respond.choices[0].message.content or {}))
    # print(history[-1]['content'])
    return respond

In [19]:
########## NOTICE ###########
# 요약: 이 셀은 안돌려도 됩니다.
# 프로젝트 백서를 간단하게 요약하는 셀 -> 이미 요약txt를 OUTPUT_summarized_project.txt에 뽑아놔서 돌릴 필요 없음
# 이후 새로운 프로젝트 백서가 쌓이면 SOURCE_completed_project_contents.txt에 추가된 프로젝트의 주제와 진행프로세스만 따로 뽑아서 붙여넣어주면 됩니다.


import os

API_KEY = os.environ["OPENAI_API_KEY"]

reference = open("./SOURCE_completed_project_contents.txt", 'r', encoding='UTF8').read()
references = reference.split('[기업명]')[1:]
prompt_sys = open("./PROMPT_summarized_project.txt", 'r', encoding='UTF8').read()
prompt_usr = """
다음 내용을 바탕으로 기업 정보를 요약해줘.

{reference}

"""
ret = []
f= open("OUTPUT_summarized_project.txt","w+", encoding='UTF8')
    
for reference in references:
    prompt = [{"role":"system", "content":prompt_sys},
                {"role":"user", "content":prompt_usr.format(reference = reference)}
            ]
    answer = run_chatgpt(API_KEY, prompt)
    ret.append(answer.choices[0].message.content)
    print(ret[-1])
    f.write(ret[-1])
    print("\n\n\n")
    f.write("\n\n\n")


f.close()



떠리몰

1. 추천시스템 개발 및 구축
- AWS Personalize 기반 개인화 추천시스템 구축
- 6가지 종류의 Recommender와 필터 기능을 가진 Campaign 구축
- 특정 조건을 만족하는 상품만을 불러오는 ‘필터’ 기능 구축

2. 구매 데이터 분석을 토대로 추천시스템 활용 가능성 제안
- 구매 횟수에 따른 고객 그룹별 구매 경향 분석
- 가격, 상품 카테고리, 할인율 중심으로 고객 분포와 선호도 분석
- 인사이트 기반 근거와 필터 기능을 활용한 구체적 추천시스템 활용 방안 제안




리플에이아이

1. 리플AI 신사업의 초기 추천시스템 구축에 필요한 데이터 스키마 제안
- 태그 기반 숏폼 추천에 적합한 모형(LightFM, NewsRec) 조사 및 MovieLens 데이터 활용 예시 모형 적합성 평가
- 모형별 Input 데이터 스키마 제안

2. AWS Personalize 기반 상용 추천시스템 구축방안 제시
- 공식 Documentation과 Demo를 통한 AWS 용어 및 워크플로우 이해
- 회사 AWS 계정 내 실시간 추천시스템 구축 및 활용 예시 웹페이지 생성
- AWS 요구 조건 및 숏폼 도메인 특성에 근거한 데이터 스키마 제안




클래스팅

1. AI 서비스의 사용자 성취도 평가 메트릭 개발
- Elo 기반 정답률 예측 모델 설계 및 유저, 문항 ranking화
- Association rule과 lift 기반 추천 방식 제언
- 찍기 detection 및 rule-based 추천 방식 제언

2. AI 서비스의 문항 추천 알고리즘 개발
- IRT 모델링을 통한 문항 변별도 분석 및 적절한 지표 선정
- 변별도와 난이도를 이용한 문제 필터링
- 오답 매력도 분석을 통한 난이도의 다른 지표 제공 및 인사이트 도출




윌라

프로젝트 주제1: 컨텐츠 추천시스템 구현
- CBF, CF, hybrid 모델, Elo rating을 활용한 bestseller 모델, 딥러닝 기반 DNN, SR-GNN, SASRec 모델 등 다양한 

In [25]:
############## NOTICE ################
# 이 셀은 일부분 수정해서 돌려야 합니다.
# TODO
# 1. 기업 맞춤형 메일을 작성해야하므로 타겟 기업에 대한 정보를 글로 긁어와야함. 단순히 투자유치 받았다 이런 기사는 쓸모없고, 기업이 어떤 문제를 어떤 접근방식으로 풀고있는지에 대한 정보가 중요함.
# --> SOURCE_target_company_information.txt에 붙여넣어주세요.
# 2. 기업명과 도메인은 직접 입력해주세요. gpt한테 추론하도록 시켰는데 기업명은 잘 잡아내지만 도메인 분류는 잘 못해서 수동입력해주는 게 좋을 듯
# --> 아래 TODO 에도 적어놨는데, prompt_usr 중 '기업명'과 '도메인'을 적절하게 수정하면 됩니다.
# 완성된 제안서는 OUTPUT_completed_proposal_letter.txt에서 확인 가능합니다.


import os

API_KEY = os.environ["OPENAI_API_KEY"]

summarized_projects = open("./OUTPUT_summarized_project.txt", 'r', encoding='UTF8').read()
template = open("./SOURCE_proposal_template.txt", 'r', encoding='UTF8').read()
target_company_information = open("./SOURCE_target_company_information.txt", 'r', encoding='UTF8').read()
prompt_sys = open("./PROMPT_complete_proposal_letter.txt", 'r', encoding='UTF8').read()

############## TODO ####################
# 이하 prompt_usr에서 '기업명'과 '도메인'을 적절하게 수정하기

prompt_usr = """
다음 내용을 바탕으로 완전한 협업 제안 메일을 발송해줘.

[타겟 기업 관련 정보]

기업명: 데이블

도메인: 광고/마케팅

{target_company_information}


[GrowthHackers에서 지난 여러 기업과 진행한 프로젝트들]

{summarized_projects}


[제안서 템플릿]

{template}


"""
ret = []
f= open("OUTPUT_completed_proposal_letter.txt","w+", encoding='UTF8')
    
prompt = [{"role":"system", "content":prompt_sys},
            {"role":"user", "content":prompt_usr.format(target_company_information = target_company_information, summarized_projects = summarized_projects, template = template)}
        ]
answer = run_chatgpt(API_KEY, prompt)
f.write(answer.choices[0].message.content)

f.close()